# Drinking water potability

Primeiramente importaremos e mostraremos como se comporta a base.

In [1]:
import pandas as pd

df = pd.read_csv("water_potability.csv")

df.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


## Tratamento dos dados

Como podemos ver abaixo existem vários valores `NaN`, *not a number*, na base. Portanto precisaremos tratar os dados, e removendo os valores nulos. Depois realizamos uma técnica conhecida como *undersampling*, na qual igualamos o número de instâncias das duas classes removendo as instâncias da classe com mais valores.

In [2]:
df.isnull().sum()

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64

In [3]:
df = df.dropna()
df.isnull().sum()

ph                 0
Hardness           0
Solids             0
Chloramines        0
Sulfate            0
Conductivity       0
Organic_carbon     0
Trihalomethanes    0
Turbidity          0
Potability         0
dtype: int64

In [4]:
import numpy as np

not_potable_indices = df[df.Potability == 0].index
number_of_potables = df[df.Potability == 1].shape[0]
random_indices = np.random.choice(not_potable_indices, number_of_potables, replace=False)
df_not_potables = df.loc[random_indices]
df_potables = df[df.Potability == 1]
df = pd.concat([df_potables, df_not_potables])

df.Potability.value_counts()

0    811
1    811
Name: Potability, dtype: int64

## Dividindo os dados em treino e teste
Dividimos os dados em `X`, sendo os instâncias e `y` os valores da classe "Potability".

In [5]:
from sklearn.model_selection import train_test_split

X = df.drop('Potability',axis=1).values
y = df['Potability'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0xB33F)

# Treinos e testes
## Bernoulli Naive Bayes

In [15]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, accuracy_score

BNB = BernoulliNB()
BNB.fit(X_train, y_train)

y_predicted = BNB.predict(X_test)

target_names = ['Not Potable', 'Potable']
print(f"Accuracy: {accuracy_score(y_test, y_predicted)}")
print(classification_report(y_test, y_predicted, target_names=target_names))

Accuracy: 0.4804928131416838
              precision    recall  f1-score   support

 Not Potable       0.48      1.00      0.65       234
     Potable       0.00      0.00      0.00       253

    accuracy                           0.48       487
   macro avg       0.24      0.50      0.32       487
weighted avg       0.23      0.48      0.31       487



/home/negreiros/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/negreiros/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/negreiros/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Análise
É interessante notar que o Bernoulli Naive Bayes não conseguiu generalizar bem, classificando todos as instâncias como 0, não potáveis. Aqui nós ainda temos que descobrir o porquê (ou não) do que isso aconteceu pra colocar no artigo. Como não existem praticamente nenhum parâmetro

## Random Forest - Conjunto de parâmetros 1

In [16]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators=100, min_samples_leaf=2, random_state=0xB33F)
RF.fit(X_train, y_train)

y_predicted = RF.predict(X_test)

target_names = ['Not Potable', 'Potable']
print(f"Accuracy: {accuracy_score(y_test, y_predicted)}")
print(classification_report(y_test, y_predicted, target_names=target_names))

Accuracy: 0.6098562628336756
              precision    recall  f1-score   support

 Not Potable       0.59      0.64      0.61       234
     Potable       0.64      0.58      0.61       253

    accuracy                           0.61       487
   macro avg       0.61      0.61      0.61       487
weighted avg       0.61      0.61      0.61       487



### Análise
Ao contrário do Bernoulli Naive Bayes, o Random Forest Classifier conseguiu generalizar para as duas classes


## Random Forest - Conjunto de parâmetros 2

In [26]:
RF = RandomForestClassifier(n_estimators=1000, min_samples_leaf=2, random_state=0xB33F)
RF.fit(X_train, y_train)

y_predicted = RF.predict(X_test)

target_names = ['Not Potable', 'Potable']
print(f"Accuracy: {accuracy_score(y_test, y_predicted)}")
print(classification_report(y_test, y_predicted, target_names=target_names))

Accuracy: 0.6262833675564682
              precision    recall  f1-score   support

 Not Potable       0.60      0.66      0.63       234
     Potable       0.66      0.59      0.62       253

    accuracy                           0.63       487
   macro avg       0.63      0.63      0.63       487
weighted avg       0.63      0.63      0.63       487



### Análise
Como podemos ver, o número de árvores da flores não influencia tanto na melhoria do modelo. Com 10 vezes o número mair de árvores, a melhor no *f1-score* é de apenas 0.01.

## Random Forest - Conjunto de parâmetros 3

In [30]:
RF = RandomForestClassifier(n_estimators=100, min_samples_leaf=1, random_state=0xB33F)
RF.fit(X_train, y_train)

y_predicted = RF.predict(X_test)

target_names = ['Not Potable', 'Potable']
print(f"Accuracy: {accuracy_score(y_test, y_predicted)}")
print(classification_report(y_test, y_predicted, target_names=target_names))

Accuracy: 0.6119096509240246
              precision    recall  f1-score   support

 Not Potable       0.58      0.66      0.62       234
     Potable       0.64      0.57      0.60       253

    accuracy                           0.61       487
   macro avg       0.61      0.61      0.61       487
weighted avg       0.62      0.61      0.61       487



### Análise
Diminuir o número de instâncias necessárias para que se construa um nó da árvore não constituiu efeito significativo.